# CANDO 
## Computer Aided Nanomaterial Design and Optimization
<ins>[CANDO](https://github.com/cando-developers/cando)</ins>  |  <ins>[Amber](https://ambermd.org/)</ins>  |  <ins>[ThirdLaw Technologies](https://www.thirdlaw.tech/)</ins>
                                                   
#### Demo 2 - Build an alanine residue, minimize, and view trajectory. <br>
 - #### This JupyterLab notebook is an introduction to building small molecules using CANDO LEAP.
 - #### We then show some energy minimization techniques and view a minimization trajectory.
 
<hr style="border:1px solid black"> </hr>

<div class="alert alert-block alert-info">
    <b>NOTE:</b> <font color='black'> you can always type "help" into a cell to see CANDO LEAP commands:</font></div>

In [ ]:
help

<div class="alert alert-block alert-info">
    <b>Step 1:</b> <font color='black'>Let's setup our Amber environment:</font></div>

In [ ]:
(setup-amber-paths)

<div class="alert alert-block alert-info">
    <b>Step 2:</b> <font color='black'>JupyterLab can load any .png file. Here is the amino acud Alanine:</font></div>

In [ ]:
(jupyter:png-file "data/alanine/alanine.png")

<div class="alert alert-block alert-info">
    <b>Step 3:</b> <font color='black'>Let's instead load the residue from a saved ChemDraw sketch. We. call the loaded residue an "aggregate":</font></div>

In [ ]:
agg = (load-chem-draw-aggregate "data/alanine/alanine2.cdxml")

<div class="alert alert-block alert-info">
    <b>Step 4:</b> <font color='black'>Since we are building from a ChemDraw sketch we need to load an AMBER force-filed, the antechamber rules for automatically assigning AMBER atom types:</font></div>

In [ ]:
(source "leaprc.protein.ff14SB")
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")
(load-atom-type-rules "data/alanine/water_amber.def")

<div class="alert alert-block alert-info">
    <b>Step 5:</b> <font color='black'>We need to manually set the stereocenters:</font></div>

In [ ]:
stereocenters = (cando:gather-stereocenters agg)

In [ ]:
(set-stereoisomer-func stereocenters (lambda (c) :R))

<div class="alert alert-block alert-info">
    <b>Step 6:</b> <font color='black'>Setup an implicit solvent box of 50 A^3:</font></div>

In [ ]:
(chem:set-property agg :solvent-box '(50.0 50.0 50.0))
(chem:fill-in-implicit-hydrogens agg)
(build-unbuilt-hydrogens agg)

<div class="alert alert-block alert-info">
    <b>Note:</b> <font color='black'>At this point we still have a planar molecule.  Let's use "show" to look at it:</font></div>

In [ ]:
(show agg)

<div class="alert alert-block alert-info">
    <b>Step 7:</b> <font color='black'>Let's save a mol2 file for later, load the GAFF force-field and "jostle" things up a bit. Then minimize our built aggregate Alanine:</font></div>

In [ ]:
(save-mol2 agg "alanine2.mol2")
(easy-gaff)
(jostle agg)
(energy:minimize agg)

<div class="alert alert-block alert-info">
    <b>Step 8:</b> <font color='black'>Use "show" to show our minimized alanine:</font></div>

In [ ]:
(show agg)

<div class="alert alert-block alert-info">
    <b>Step 9:</b> <font color='black'>Now that we have a 3D molecule we can setup: 1) an energy function 2) a minimizer, and 3) a trajectory data structure:</font></div>

In [ ]:
energy = (chem:make-energy-function :matter agg)

In [ ]:
min = (chem:make-minimizer energy)

In [ ]:
traj = (make-instance 'cando-trajectory :matter agg)

In [ ]:
(chem:set-step-callback min (lambda (coords) (let ((arr (make-array (length coords) :element-type 'single-float)))
                                                    (loop for idx below (length coords)
                                                          do (setf (aref arr idx) (float (aref coords idx) 1s0)))
                                                            (nglv:append-coordinates traj arr))))

<div class="alert alert-block alert-info">
    <b>Step 10:</b> <font color='black'>Run the minimization:</font></div>

In [ ]:
(energy:minimize-minimizer min)

In [ ]:
(show traj)

In [ ]:
(leap.topology:save-amber-parm-format agg "agg.top" "agg.crd")

In [ ]:
startSwank